In [ ]:
import anthropic
import mu2e
import json
import os

client = anthropic.Anthropic(api_key="")

In [ ]:
#generating first system prompt
def assignPrompt1(q):

        website_content_string = ""
        for doc in mu2e.findAndGet(q):
            website_content_string += "<page url=\""+doc['url']+"\" title=\""+doc['heading']+"\">\n"+doc['text']+"</page>\n"

        SYSTEM_PROMPT = f"""You are a mu2e DocBot, a helpful assistant that is an expert at helping users with the mu2e related questions.

        Here is some related Mu2e documentation
	<documentation>
	{website_content_string}
	</documentation>

        When a user asks a question, peform the following tasks:
        1. Find sections in the documentation that are the most relevant to answering the question. You may need to use multiple pages.
        2. Keep track of the url of the pages.
        3. Based on the document, answer the question. Directly quote the documentation when possible, including examples.
        4.  When answering the question provide references to the corresponding urls in square brackets from step 2.
        Format the answer as plain text, formatted as Markdown[1]
	"""
        return SYSTEM_PROMPT


#second
def assignPrompt2(q):

        website_content_string = ""
        for doc in mu2e.findAndGet(q):
            website_content_string += "<page url=\""+doc['url']+"\" title=\""+doc['heading']+"\">\n"+doc['text']+"</page>\n"

        SYSTEM_PROMPT = f"""You are a mu2e DocBot, a helpful assistant that is an expert at helping users with the mu2e related questions.

        Here is some related Mu2e documentation
	<documentation>
	{website_content_string}
	</documentation>

    Give a concise, accurate answer using the information provided.
    
	"""
        return SYSTEM_PROMPT

#third
def assignPrompt3(q):

        website_content_string = ""
        for doc in mu2e.findAndGet(q):
            website_content_string += "<page url=\""+doc['url']+"\" title=\""+doc['heading']+"\">\n"+doc['text']+"</page>\n"

        SYSTEM_PROMPT = f"""You are a mu2e DocBot, a helpful assistant that is an expert at helping users with the mu2e related questions.

        Here is some related Mu2e documentation
	<documentation>
	{website_content_string}
	</documentation>
    
	"""
        return SYSTEM_PROMPT


In [3]:
def ask_claude(q, system_prompt):
    message = client.messages.create(
    model = "claude-3-haiku-20240307",
    system = system_prompt,
    max_tokens = 512,
    messages = [
        {"role":"user", "content": q }
        ]
    )
    return message.content[0].text

In [ ]:
#Example: q = "What is the primary purpose of the Mu2e experiment?"
response1 = ask_claude(q, assignPrompt1(q))
response2 = ask_claude(q, assignPrompt2(q))
response3 = ask_claude(q, assignPrompt3(q))

In [ ]:
website_content_string = ""
for doc in mu2e.findAndGet(q):
    website_content_string += "<page url=\""+doc['url']+"\" title=\""+doc['heading']+"\">\n"+doc['text']+"</page>\n"

message = client.messages.create(
    model = "claude-3-haiku-20240307",
    max_tokens = 300,
    messages = [
        {"role":"user", "content": f'''
        
        I asked you: {q}
        
        I got these three responses:

        first response = {response1}
        second response = {response2}
        third response = {response3}

        The data these responses are based on can be found:
        <documentation>
    	{website_content_string}
    	</documentation>

        Which response is the best? Rank the three responses.'''
        
        
        }
    ]
)
print(message.content[0].text)

In [ ]:
#finding cosine similarity
from transformers import AutoTokenizer
from angle_emb import AnglE
angle = AnglE.from_pretrained('WhereIsAI/UAE-Large-V1', pooling_strategy='cls').cuda()

embeddingResponse1 = angle.encode([response1])[0]
embeddingResponse2 = angle.encode([response2])[0]
embeddingResponse3 = angle.encode([response3])[0]

embeddingResponse1 = embeddingResponse1.reshape(1, -1)
embeddingResponse2 = embeddingResponse2.reshape(1, -1)
embeddingResponse3 = embeddingResponse3.reshape(1, -1)

embeddingQuestion = angle.encode([q])[0]
embeddingQuestion = embeddingQuestion.reshape(1, -1)

embeddingPrompt1 = angle.encode([assignPrompt(q)])[0]
embeddingPrompt2 = angle.encode([assignPrompt2(q)])[0]
embeddingPrompt3 = angle.encode([assignPrompt3(q)])[0]

embeddingPrompt1 = embeddingPrompt1.reshape(1, -1)
embeddingPrompt2 = embeddingPrompt2.reshape(1, -1)
embeddingPrompt3 = embeddingPrompt3.reshape(1, -1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosineSimP1 = cosine_similarity(embeddingQuestion, embeddingPrompt1)[0][0]
cosineSimP2 = cosine_similarity(embeddingQuestion, embeddingPrompt2)[0][0]
cosineSimP3 = cosine_similarity(embeddingQuestion, embeddingPrompt3)[0][0]
cosineSimR1 = cosine_similarity(embeddingQuestion, embeddingResponse1)[0][0]
cosineSimR2 = cosine_similarity(embeddingQuestion, embeddingResponse2)[0][0]
cosineSimR3 = cosine_similarity(embeddingQuestion, embeddingResponse3)[0][0]

print(cosineSimP1)
print(cosineSimP2)
print(cosineSimP3)
print(cosineSimR1)
print(cosineSimR2)
print(cosineSimR3) 

In [ ]:
#finding rouge score
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

candidate_summaries = [response1, response2, response3]
reference = mu2e.findAndGet(q)
reference_summary = ""
for e in reference:
    reference_summary += str(e)

scores = {key: [] for key in ['rouge1', 'rouge2', 'rougeL']}
for can in candidate_summaries:
    temp_scores = scorer.score(reference_summary, can)
    for key in temp_scores:
        scores[key].append(temp_scores[key])

for key in scores:
    print(f'{key}:\n{scores[key]}')

In [ ]:
#finding BERTScore

# Step 1: Import the required libraries
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Step 2: Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Step 3: Define the two texts to compare

def find_BERTScore(reference, response):
    text1 = text1
    text2 = text2
#text1 = reference_summary

#responses = [response1, response2, response3]
#text2 = response you are comparing
#text2 = response2

    # Step 4: Prepare the texts for BERT
    inputs1 = tokenizer(text1, return_tensors="pt", padding=True, truncation=True)
    inputs2 = tokenizer(text2, return_tensors="pt", padding=True, truncation=True)

    # Step 5: Feed the texts to the BERT model
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

    # Step 6: Obtain the representation vectors
    embeddings1 = outputs1.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings2 = outputs2.last_hidden_state.mean(dim=1).detach().numpy()

    # Step 7: Calculate cosine similarity
    similarity = np.dot(embeddings1, embeddings2.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))

    # Step 8: Print the result
    print("Similarity between the texts: {:.4f}".format(similarity[0][0]))


In [ ]:
responses = [response1, response2, response3]

for response in responses:
    print("Calculating BERTScore between reference text and response.")
    find_BERTScore(reference_summary, response)